# bigleaf test

[tutorial](https://cran.r-project.org/web/packages/bigleaf/vignettes/bigleaf_tutorial.pdf)

[document](https://cran.r-project.org/web/packages/bigleaf/bigleaf.pdf)

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
install.packages("bigleaf")
library(bigleaf)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘DEoptimR’, ‘robustbase’, ‘solartime’

trying URL 'https://cran.rstudio.com/src/contrib/DEoptimR_1.1-3-1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/robustbase_0.99-4-1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/solartime_0.0.4.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/bigleaf_0.8.2.tar.gz'

The downloaded source packages are in
	‘/tmp/RtmpC88flR/downloaded_packages’


### bigleaf-p1

In [3]:
import numpy as np
import pandas as pd
from packaging import version

def check_input(data, vars_list):
    """
    Check input data for validity and consistency.

    Args:
        data: DataFrame or None, input data
        vars_list: List of variables (column names or numeric vectors)

    Raises:
        ValueError: If input variables are invalid or inconsistent
    """
    if data is None:
        data = pd.DataFrame()

    # varnames = [v if isinstance(v, str) else v.__name__ for v in vars_list]
    varnames = [v if isinstance(v, str) else getattr(v, '__name__', str(v)) for v in vars_list]

    for v, var in enumerate(vars_list):
        varname = varnames[v]

        if isinstance(var, str):
            if not data.empty and var in data.columns:
                if np.issubdtype(data[var].dtype, np.number):
                    globals()[varname] = data[var]
                else:
                    raise ValueError(f"Column '{varname}' in input must be numeric")
            else:
                raise ValueError(f"No column named '{var}' in input for variable '{varname}'")
        else:
            if not data.empty:
                if np.issubdtype(np.array(var).dtype, np.number) and len(var) == len(data):
                    globals()[varname] = var
                elif np.issubdtype(np.array(var).dtype, np.number) and len(var) == 1:
                    globals()[varname] = np.repeat(var, len(data))
                else:
                    raise ValueError(f"Variable '{varname}' must match input length or be length 1")
            else:
                if np.issubdtype(np.array(var).dtype, np.number):
                    globals()[varname] = var
                else:
                    raise ValueError(f"Variable '{varname}' must be numeric")

def LE_to_ET(LE, Tair):
    """
    Convert latent heat flux to evapotranspiration.

    Args:
        LE: Latent heat flux (W m-2)
        Tair: Air temperature (degC)

    Returns:
        ET: Evapotranspiration (kg H2O m-2 s-1)
    """
    lambda_ = latent_heat_vaporization(Tair)
    ET = LE / lambda_
    return ET

def latent_heat_vaporization(Tair):
    """
    Calculate latent heat of vaporization.

    Args:
        Tair: Air temperature (degC)

    Returns:
        lambda_: Latent heat of vaporization (J kg-1)
    """
    k1 = 2.501
    k2 = 0.00237
    lambda_ = (k1 - k2 * Tair) * 1e6
    return lambda_

def WUE_metrics(data, GPP="GPP", NEE="NEE", LE="LE", VPD="VPD", Tair="Tair", constants=None):
    """
    Calculate water use efficiency metrics.

    Args:
        data: DataFrame with required columns
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        NEE: Column name for net ecosystem exchange (umol CO2 m-2 s-1)
        LE: Column name for latent heat flux (W m-2)
        VPD: Column name for vapor pressure deficit (kPa)
        Tair: Column name for air temperature (degC)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        dict: WUE, WUE_NEE, IWUE, uWUE metrics
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [GPP, NEE, LE, VPD, Tair])

    ET = LE_to_ET(data[LE], data[Tair])  # kg H2O m-2 s-1
    GPP_val = (data[GPP] * constants['umol2mol'] * constants['Cmol']) * constants['kg2g']  # gC m-2 s-1
    NEE_val = (data[NEE] * constants['umol2mol'] * constants['Cmol']) * constants['kg2g']  # gC m-2 s-1

    WUE = np.nanmedian(GPP_val / ET)
    WUE_NEE = np.nanmedian(np.abs(NEE_val) / ET)
    IWUE = np.nanmedian((GPP_val * data[VPD]) / ET)
    uWUE = np.nanmedian((GPP_val * np.sqrt(data[VPD])) / ET)

    return {'WUE': WUE, 'WUE_NEE': WUE_NEE, 'IWUE': IWUE, 'uWUE': uWUE}

def Gb_Thom(ustar, Sc=None, Sc_name=None, constants=None):
    """
    Calculate boundary layer conductance using Thom (1972) model.

    Args:
        ustar: Friction velocity (m s-1)
        Sc: Schmidt numbers (optional)
        Sc_name: Names for Schmidt numbers (optional)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Gb_h, Rb_h, kB_h, and Gb_x for CO2 and other scalars
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(None, [ustar])

    Rb_h = 6.2 * ustar ** -0.667
    Gb_h = 1 / Rb_h
    kB_h = Rb_h * constants['k'] * ustar

    if Sc is not None or Sc_name is not None:
        if len(Sc) != len(Sc_name):
            raise ValueError("Arguments 'Sc' and 'Sc_name' must have the same length")
        if not all(isinstance(x, (int, float)) for x in Sc):
            raise ValueError("Argument 'Sc' must be numeric")

    Sc = [constants['Sc_CO2']] + (Sc if Sc is not None else [])
    Gb_x = pd.DataFrame({f"Gb_{name}": Gb_h / (sc / constants['Pr']) ** 0.67
                         for sc, name in zip(Sc, ['CO2'] + (Sc_name if Sc_name else []))})

    return pd.DataFrame({'Gb_h': Gb_h, 'Rb_h': Rb_h, 'kB_h': kB_h, **Gb_x})

def Gb_Choudhury(data, Tair="Tair", pressure="pressure", wind="wind", ustar="ustar", H="H",
                 leafwidth=None, LAI=None, zh=None, zr=None, d=None, z0m=None,
                 stab_formulation="Dyer_1970", Sc=None, Sc_name=None, constants=None):
    """
    Calculate boundary layer conductance using Choudhury (1988) model.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        wind: Column name for wind speed (m s-1)
        ustar: Column name for friction velocity (m s-1)
        H: Column name for sensible heat flux (W m-2)
        leafwidth: Leaf width (m)
        LAI: Leaf area index
        zh: Canopy height (m)
        zr: Reference height (m)
        d: Displacement height (m)
        z0m: Roughness length for momentum (m)
        stab_formulation: Stability formulation ('Dyer_1970' or 'Businger_1971')
        Sc: Schmidt numbers (optional)
        Sc_name: Names for Schmidt numbers (optional)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Gb_h, Rb_h, kB_h, and Gb_x for CO2 and other scalars
    """
    if constants is None:
        constants = bigleaf_constants()

    if stab_formulation not in ["Dyer_1970", "Businger_1971"]:
        raise ValueError("stab_formulation must be 'Dyer_1970' or 'Businger_1971'")

    check_input(data, [Tair, pressure, wind, ustar, H])

    alpha = 4.39 - 3.97 * np.exp(-0.258 * LAI)
    estimate_z0m = z0m is None

    wind_zh = wind_profile(data, z=zh, Tair=Tair, pressure=pressure, ustar=ustar, H=H,
                           wind=wind, zr=zr, zh=zh, d=d, z0m=z0m, estimate_z0m=estimate_z0m,
                           stab_correction=True, stab_formulation=stab_formulation, constants=constants)

    wind_zh = np.maximum(0.01, wind_zh)

    if Sc is not None or Sc_name is not None:
        if len(Sc) != len(Sc_name):
            raise ValueError("Arguments 'Sc' and 'Sc_name' must have the same length")
        if not all(isinstance(x, (int, float)) for x in Sc):
            raise ValueError("Argument 'Sc' must be numeric")

    Gb_h = LAI * ((0.02 / alpha) * np.sqrt(wind_zh / leafwidth) * (1 - np.exp(-alpha / 2)))
    Rb_h = 1 / Gb_h
    kB_h = Rb_h * constants['k'] * data[ustar]

    Sc = [constants['Sc_CO2']] + (Sc if Sc is not None else [])
    Gb_x = pd.DataFrame({f"Gb_{name}": Gb_h / (sc / constants['Pr']) ** 0.67
                         for sc, name in zip(Sc, ['CO2'] + (Sc_name if Sc_name else []))})

    return pd.DataFrame({'Gb_h': Gb_h, 'Rb_h': Rb_h, 'kB_h': kB_h, **Gb_x})

def Gb_Su(data, Tair="Tair", pressure="pressure", ustar="ustar", wind="wind", H="H",
          zh=None, zr=None, d=None, z0m=None, Dl=None, fc=None, LAI=None, N=2, Cd=0.2, hs=0.01,
          stab_formulation="Dyer_1970", Sc=None, Sc_name=None, constants=None):
    """
    Calculate boundary layer conductance using Su (2001) model.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        ustar: Column name for friction velocity (m s-1)
        wind: Column name for wind speed (m s-1)
        H: Column name for sensible heat flux (W m-2)
        zh: Canopy height (m)
        zr: Reference height (m)
        d: Displacement height (m)
        z0m: Roughness length for momentum (m)
        Dl: Leaf dimension (m)
        fc: Fractional canopy coverage
        LAI: Leaf area index
        N: Number of leaf sides
        Cd: Drag coefficient
        hs: Soil roughness length (m)
        stab_formulation: Stability formulation ('Dyer_1970' or 'Businger_1971')
        Sc: Schmidt numbers (optional)
        Sc_name: Names for Schmidt numbers (optional)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Gb_h, Rb_h, kB_h, and Gb_x for CO2 and other scalars
    """
    if constants is None:
        constants = bigleaf_constants()

    if stab_formulation not in ["Dyer_1970", "Businger_1971"]:
        raise ValueError("stab_formulation must be 'Dyer_1970' or 'Businger_1971'")

    check_input(data, [Tair, pressure, ustar, wind, H])

    if fc is None:
        if LAI is None:
            raise ValueError("One of 'fc' or 'LAI' must be provided")
        fc = 1 - np.exp(-LAI / 2)

    estimate_z0m = z0m is None

    wind_zh = wind_profile(data, z=zh, Tair=Tair, pressure=pressure, ustar=ustar, H=H,
                           wind=wind, zr=zr, zh=zh, d=d, z0m=z0m, estimate_z0m=estimate_z0m,
                           stab_correction=True, stab_formulation=stab_formulation, constants=constants)

    v = kinematic_viscosity(data[Tair], data[pressure], constants)
    Re = Reynolds_Number(data[Tair], data[pressure], data[ustar], hs, constants)
    kBs = 2.46 * Re ** 0.25 - np.log(7.4)
    Reh = Dl * wind_zh / v
    Ct = 1 * constants['Pr'] ** -0.6667 * Reh ** -0.5 * N

    kB_h = (constants['k'] * Cd) / (4 * Ct * data[ustar] / wind_zh) * fc ** 2 + kBs * (1 - fc) ** 2
    Rb_h = kB_h / (constants['k'] * data[ustar])
    Gb_h = 1 / Rb_h

    if Sc is not None or Sc_name is not None:
        if len(Sc) != len(Sc_name):
            raise ValueError("Arguments 'Sc' and 'Sc_name' must have the same length")
        if not all(isinstance(x, (int, float)) for x in Sc):
            raise ValueError("Argument 'Sc' must be numeric")

    Sc = [constants['Sc_CO2']] + (Sc if Sc is not None else [])
    Gb_x = pd.DataFrame({f"Gb_{name}": Gb_h / (sc / constants['Pr']) ** 0.67
                         for sc, name in zip(Sc, ['CO2'] + (Sc_name if Sc_name else []))})

    return pd.DataFrame({'Gb_h': Gb_h, 'Rb_h': Rb_h, 'kB_h': kB_h, **Gb_x})

def roughness_length_heat(z0m, kB_h):
    """
    Calculate roughness length for heat.

    Args:
        z0m: Roughness length for momentum (m)
        kB_h: Logarithmic ratio of z0m to z0h

    Returns:
        z0h: Roughness length for heat (m)
    """
    z0h = z0m / np.exp(kB_h)
    return z0h

def aerodynamic_conductance(data, Tair="Tair", pressure="pressure", wind="wind", ustar="ustar", H="H",
                            zr=None, zh=None, d=None, z0m=None, Dl=None, N=2, fc=None, LAI=None,
                            Cd=0.2, hs=0.01, wind_profile=False, stab_correction=True,
                            stab_formulation="Dyer_1970", Rb_model="Thom_1972", kB_h=None,
                            Sc=None, Sc_name=None, constants=None):
    """
    Calculate aerodynamic conductance for momentum and scalars.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        wind: Column name for wind speed (m s-1)
        ustar: Column name for friction velocity (m s-1)
        H: Column name for sensible heat flux (W m-2)
        zr: Reference height (m)
        zh: Canopy height (m)
        d: Displacement height (m)
        z0m: Roughness length for momentum (m)
        Dl: Leaf dimension (m)
        N: Number of leaf sides
        fc: Fractional canopy coverage
        LAI: Leaf area index
        Cd: Drag coefficient
        hs: Soil roughness length (m)
        wind_profile: Boolean, use wind profile for Ga_m
        stab_correction: Boolean, apply stability correction
        stab_formulation: Stability formulation ('Dyer_1970' or 'Businger_1971')
        Rb_model: Boundary layer model ('Thom_1972', 'Choudhury_1988', 'Su_2001', 'constant_kB-1')
        kB_h: Constant kB-1 value (if Rb_model='constant_kB-1')
        Sc: Schmidt numbers (optional)
        Sc_name: Names for Schmidt numbers (optional)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Ga_m, Ra_m, Ga_h, Ra_h, Gb_h, Rb_h, kB_h, z0h, zeta, psi_h, Ra_CO2, and other conductances
    """
    if constants is None:
        constants = bigleaf_constants()

    pv = version.parse("0.7.5")  # Placeholder for package version
    if pv > version.parse("0.7.5"):
        print("Note new column 'z0h' in the function output for 'bigleaf' versions > 0.7.5.")

    if Rb_model not in ["Thom_1972", "Choudhury_1988", "Su_2001", "constant_kB-1"]:
        raise ValueError("Rb_model must be one of 'Thom_1972', 'Choudhury_1988', 'Su_2001', 'constant_kB-1'")
    if stab_formulation not in ["Dyer_1970", "Businger_1971"]:
        raise ValueError("stab_formulation must be 'Dyer_1970' or 'Businger_1971'")

    check_input(data, [Tair, pressure, wind, ustar, H])

    if Rb_model in ["Thom_1972", "Choudhury_1988", "Su_2001"]:
        if Rb_model == "Thom_1972":
            Gb_mod = Gb_Thom(ustar=data[ustar], Sc=Sc, Sc_name=Sc_name, constants=constants)
        elif Rb_model == "Choudhury_1988":
            Gb_mod = Gb_Choudhury(data, Tair=Tair, pressure=pressure, wind=wind, ustar=ustar,
                                  H=H, leafwidth=Dl, LAI=LAI, zh=zh, zr=zr, d=d, z0m=z0m,
                                  stab_formulation=stab_formulation, Sc=Sc, Sc_name=Sc_name,
                                  constants=constants)
        else:  # Su_2001
            Gb_mod = Gb_Su(data, Tair=Tair, pressure=pressure, ustar=ustar, wind=wind,
                           H=H, zh=zh, zr=zr, d=d, z0m=z0m, Dl=Dl, fc=fc, LAI=LAI,
                           Cd=Cd, hs=hs, stab_formulation=stab_formulation, Sc=Sc,
                           Sc_name=Sc_name, constants=constants)

        kB_h = Gb_mod['kB_h']
        Rb_h = Gb_mod['Rb_h']
        Gb_h = Gb_mod['Gb_h']
        Gb_x = Gb_mod[[col for col in Gb_mod.columns if col.startswith('Gb_') and col != 'Gb_h']]

    else:  # constant_kB-1
        if kB_h is None:
            raise ValueError("kB_h must be specified for Rb_model='constant_kB-1'")
        Rb_h = kB_h / (constants['k'] * data[ustar])
        Gb_h = 1 / Rb_h

        if Sc is not None or Sc_name is not None:
            if len(Sc) != len(Sc_name):
                raise ValueError("Arguments 'Sc' and 'Sc_name' must have the same length")
            if not all(isinstance(x, (int, float)) for x in Sc):
                raise ValueError("Argument 'Sc' must be numeric")

        Sc = [constants['Sc_CO2']] + (Sc if Sc is not None else [])
        Gb_x = pd.DataFrame({f"Gb_{name}": Gb_h / (sc / constants['Pr']) ** 0.67
                             for sc, name in zip(Sc, ['CO2'] + (Sc_name if Sc_name else []))})

    if wind_profile:
        if z0m is None and Rb_model in ["constant_kB-1", "Thom_1972"]:
            raise ValueError("z0m must be provided if wind_profile=True")
        elif z0m is None and Rb_model in ["Choudhury_1988", "Su_2001"]:
            z0m = roughness_parameters(method="wind_profile", zh=zh, zr=zr, d=d, data=data,
                                       Tair=Tair, pressure=pressure, wind=wind, ustar=ustar, H=H,
                                       stab_roughness=True, stab_formulation=stab_formulation,
                                       constants=constants)['z0m']

        if stab_correction:
            zeta = stability_parameter(data, Tair=Tair, pressure=pressure, ustar=ustar,
                                      H=H, zr=zr, d=d, constants=constants)
            if stab_formulation in ["Dyer_1970", "Businger_1971"]:
                psi_h = stability_correction(zeta, formulation=stab_formulation)['psi_h']
            else:
                raise ValueError("stab_formulation must be 'Dyer_1970' or 'Businger_1971'")

            Ra_m = np.maximum(np.log((zr - d) / z0m) - psi_h, 0) / (constants['k'] * data[ustar])
        else:
            Ra_m = np.maximum(np.log((zr - d) / z0m), 0) / (constants['k'] * data[ustar])
            zeta = psi_h = np.full_like(Ra_m, np.nan)
    else:
        if any(x is not None for x in [zr, d, z0m]) and Rb_model in ["constant_kB-1", "Thom_1972"]:
            print("Warning: Provided roughness parameters (zr, d, z0m) are not used if wind_profile=False")
        Ra_m = data[wind] / data[ustar] ** 2
        zeta = psi_h = np.full_like(Ra_m, np.nan)

    Ga_m = 1 / Ra_m
    Ra_h = Ra_m + Rb_h
    Ga_h = 1 / Ra_h
    # Ga_x = 1 / (Ra_m + 1 / Gb_x)
    Ga_x = 1 / (Gb_x.rdiv(1).add(Ra_m, axis=0))
    Ga_x = Ga_x.rename(columns={col: col.replace('Gb_', 'Ga_') for col in Ga_x.columns})
    Ra_CO2 = 1 / Ga_x['Ga_CO2']

    if z0m is not None:
        z0h = roughness_length_heat(z0m, kB_h)
    else:
        z0h = np.full_like(Ra_m, np.nan)

    Gab_x = pd.concat([Ga_x, Gb_x], axis=1)
    Gab_x = Gab_x[[Gab_x.columns[i] for i in np.repeat([0, len(Ga_x.columns)], len(Ga_x.columns)) +
                   np.sort(np.repeat(np.arange(len(Ga_x.columns)), 2))]]

    return pd.DataFrame({
        'Ga_m': Ga_m, 'Ra_m': Ra_m, 'Ga_h': Ga_h, 'Ra_h': Ra_h, 'Gb_h': Gb_h,
        'Rb_h': Rb_h, 'kB_h': kB_h, 'z0h': z0h, 'zeta': zeta, 'psi_h': psi_h,
        'Ra_CO2': Ra_CO2, **Gab_x
    })

def bigleaf_constants():
    """
    Define physical and conversion constants used in the bigleaf package.

    Returns:
        dict: Dictionary of constants
    """
    return {
        # Physical constants
        'cp': 1004.834,          # specific heat of air (J K-1 kg-1)
        'Rgas': 8.31451,         # universal gas constant (J mol-1 K-1)
        'Rv': 461.5,             # gas constant of water vapor (J kg-1 K-1)
        'Rd': 287.0586,          # gas constant of dry air (J kg-1 K-1)
        'Md': 0.0289645,         # molar mass of dry air (kg mol-1)
        'Mw': 0.0180153,         # molar mass of water vapor (kg mol-1)
        'eps': 0.622,            # ratio of molecular weight of water vapor to dry air
        'g': 9.81,               # gravitational acceleration (m s-2)
        'solar_constant': 1366.1, # solar constant (W m-2)
        'pressure0': 101325,     # reference atmospheric pressure (Pa)
        'Tair0': 273.15,         # reference air temperature (K)
        'k': 0.41,               # von Karman constant
        'Cmol': 0.012011,        # molar mass of carbon (kg mol-1)
        'Omol': 0.0159994,       # molar mass of oxygen (kg mol-1)
        'H2Omol': 0.01801528,    # molar mass of water (kg mol-1)
        'sigma': 5.670367e-08,   # Stefan-Boltzmann constant (W m-2 K-4)
        'Pr': 0.71,              # Prandtl number
        'Sc_CO2': 1.07,          # Schmidt number for CO2
        'Le067': 0.93,           # Lewis number for water vapor to the power of 0.67
        # Conversion constants
        'Kelvin': 273.15,        # conversion degree Celsius to Kelvin
        'DwDc': 1.6,             # ratio of molecular diffusivities for water vapor and CO2
        'days2seconds': 86400,   # seconds per day
        'kPa2Pa': 1000,          # conversion kilopascal to pascal
        'Pa2kPa': 0.001,         # conversion pascal to kilopascal
        'umol2mol': 1e-06,       # conversion micromole to mole
        'mol2umol': 1e06,        # conversion mole to micromole
        'kg2g': 1000,            # conversion kilogram to gram
        'g2kg': 0.001,           # conversion gram to kilogram
        'kJ2J': 1000,            # conversion kilojoule to joule
        'J2kJ': 0.001,           # conversion joule to kilojoule
        'se_median': 1.253,      # conversion SE of mean to SE of median
        'frac2percent': 100       # conversion fraction to percent
    }

### bigleaf-p2

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm
# from .bigleaf_part1 import check_input, bigleaf_constants, LE_to_ET, latent_heat_vaporization

def intercellular_CO2(data, Ca="Ca", GPP="GPP", Gs="Gs_mol", Rleaf=None,
                     missing_Rleaf_as_NA=False, constants=None):
    """
    Calculate intercellular CO2 concentration.

    Args:
        data: DataFrame with required columns
        Ca: Column name for atmospheric CO2 concentration (umol mol-1)
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        Gs: Column name for stomatal conductance (mol m-2 s-1)
        Rleaf: Leaf respiration (umol CO2 m-2 s-1), optional
        missing_Rleaf_as_NA: If True, treat missing Rleaf as NA; else as 0
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Ci: Intercellular CO2 concentration (umol mol-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Ca, GPP, Gs])

    if Rleaf is not None:
        if not missing_Rleaf_as_NA:
            Rleaf = np.where(np.isnan(Rleaf), 0, Rleaf)
    else:
        print("Respiration from the leaves is ignored and set to 0.")
        Rleaf = 0

    Ci = data[Ca] - (data[GPP] - Rleaf) / (data[Gs] / constants['DwDc'])

    return Ci

def photosynthetic_capacity(data, C3=True, Temp="Temp", GPP="GPP", Ci="Ci", PPFD="PPFD",
                          PPFD_j=[200, 500], PPFD_c=1000, Rleaf=None, Oi=0.21,
                          Kc25=404.9, Ko25=278.4, Gam25=42.75, Kc_Ha=79.43, Ko_Ha=36.38,
                          Gam_Ha=37.83, Vcmax_Ha=65.33, Vcmax_Hd=200, Vcmax_dS=0.635,
                          Jmax_Ha=43.9, Jmax_Hd=200, Jmax_dS=0.640, Theta=0.7,
                          alpha_canopy=0.8, missing_Rleaf_as_NA=False, Ci_C4=100,
                          constants=None):
    """
    Calculate photosynthetic capacity parameters (Vcmax25, Jmax25).

    Args:
        data: DataFrame with required columns
        C3: Boolean, True for C3 plants, False for C4
        Temp: Column name for temperature (degC)
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        Ci: Column name for intercellular CO2 concentration (umol mol-1)
        PPFD: Column name for photosynthetic photon flux density (umol m-2 s-1)
        PPFD_j: PPFD range for J calculation [min, max]
        PPFD_c: Minimum PPFD for Vcmax calculation
        Rleaf: Leaf respiration (umol CO2 m-2 s-1), optional
        Oi: Oxygen concentration (mol mol-1)
        Kc25, Ko25, Gam25: Michaelis-Menten constants at 25C
        Kc_Ha, Ko_Ha, Gam_Ha: Activation energies (kJ mol-1)
        Vcmax_Ha, Vcmax_Hd, Vcmax_dS: Arrhenius parameters for Vcmax
        Jmax_Ha, Jmax_Hd, Jmax_dS: Arrhenius parameters for Jmax
        Theta: Curvature parameter for Jmax
        alpha_canopy: Canopy absorptance
        missing_Rleaf_as_NA: If True, treat missing Rleaf as NA; else as 0
        Ci_C4: Minimum Ci for C4 plants
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        dict: Vcmax25, Vcmax25_SE, Jmax25, Jmax25_SE
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Temp, GPP, Ci, PPFD])

    Temp = data[Temp] + constants['Kelvin']
    Tref = 25.0 + constants['Kelvin']
    Ci = data[Ci]

    if C3:
        Kc_Ha = Kc_Ha * constants['kJ2J']
        Ko_Ha = Ko_Ha * constants['kJ2J']
        Gam_Ha = Gam_Ha * constants['kJ2J']

        Kc = Kc25 * np.exp(Kc_Ha * (Temp - Tref) / (Tref * constants['Rgas'] * Temp))
        Ko = Ko25 * np.exp(Ko_Ha * (Temp - Tref) / (Tref * constants['Rgas'] * Temp))
        Gam = Gam25 * np.exp(Gam_Ha * (Temp - Tref) / (Tref * constants['Rgas'] * Temp))
        Ko = Ko * constants['J2kJ']

        Ci = np.where((Ci < 80) | np.isnan(Ci), np.nan, Ci)

        GPPc = GPPj = data[GPP]
        GPPj = np.where((data[PPFD] < PPFD_j[0]) | (data[PPFD] > PPFD_j[1]) | np.isnan(data[PPFD]), np.nan, GPPj)
        GPPc = np.where((data[PPFD] < PPFD_c) | np.isnan(data[PPFD]), np.nan, GPPc)

        if Rleaf is not None:
            if not missing_Rleaf_as_NA:
                Rleaf = np.where(np.isnan(Rleaf), 0, Rleaf)
        else:
            print("Respiration from the leaves is ignored and set to 0.")
            Rleaf = 0

        Vcmax = (GPPc - Rleaf) * (Ci + Kc * (1.0 + Oi / Ko)) / (Ci - Gam)
        J = (GPPj - Rleaf) * (4.0 * Ci + 8.0 * Gam) / (Ci - Gam)
    else:
        GPPc = GPPj = data[GPP]
        GPPj = np.where((data[PPFD] < PPFD_j[0]) | (data[PPFD] > PPFD_j[1]) | np.isnan(data[PPFD]) | (Ci < 0), np.nan, GPPj)
        GPPc = np.where((data[PPFD] < PPFD_c) | (Ci < Ci_C4) | np.isnan(data[PPFD]), np.nan, GPPc)

        Vcmax = GPPc
        J = 3 * GPPj / (1 - 0.5)

    APPFD_PSII = data[PPFD] * alpha_canopy * 0.85 * 0.5

    calcJmax = np.where(np.isfinite(J) & np.isfinite(APPFD_PSII))[0]
    if len(calcJmax) > 0:
        Jmax = np.array([
            minimize(
                lambda Jmax: np.abs(J[i] - (
                    (APPFD_PSII[i] + Jmax - np.sqrt((APPFD_PSII[i] + Jmax)**2 - 4.0 * Theta * APPFD_PSII[i] * Jmax)) /
                    (2.0 * Theta)
                )),
                x0=500, bounds=[(0, 1000)], tol=1e-2
            ).x[0] if np.isfinite(J[i]) else np.nan
            for i in calcJmax
        ])
    else:
        print("Warning: Not enough observations to calculate Jmax!")
        Jmax = np.nan

    Vcmax25 = Arrhenius_temp_response(Vcmax, Temp - constants['Kelvin'],
                                    Ha=Vcmax_Ha, Hd=Vcmax_Hd, dS=Vcmax_dS, constants=constants)

    Jmax25 = Arrhenius_temp_response(Jmax, Temp[calcJmax] - constants['Kelvin'],
                                    Ha=Jmax_Ha, Hd=Jmax_Hd, dS=Jmax_dS, constants=constants)

    Vcmax25_Median = np.nanmedian(Vcmax25)
    Vcmax25_SE = constants['se_median'] * np.nanstd(Vcmax25) / np.sqrt(np.sum(~np.isnan(Vcmax25)))
    Jmax25_Median = np.nanmedian(Jmax25)
    Jmax25_SE = constants['se_median'] * np.nanstd(Jmax25) / np.sqrt(np.sum(~np.isnan(Jmax25)))

    return {
        'Vcmax25': round(Vcmax25_Median, 2),
        'Vcmax25_SE': round(Vcmax25_SE, 2),
        'Jmax25': round(Jmax25_Median, 2),
        'Jmax25_SE': round(Jmax25_SE, 2)
    }

def Arrhenius_temp_response(param, Temp, Ha=None, Hd=None, dS=None, constants=None):
    """
    Calculate temperature response using Arrhenius equation.

    Args:
        param: Parameter values
        Temp: Temperature (degC)
        Ha: Activation energy (kJ mol-1)
        Hd: Deactivation energy (kJ mol-1)
        dS: Entropy term (kJ mol-1 K-1)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        param25: Parameter values normalized to 25C
    """
    if constants is None:
        constants = bigleaf_constants()

    Temp = Temp + constants['Kelvin']
    Tref = 25.0 + constants['Kelvin']

    Ha = Ha * constants['kJ2J'] if Ha is not None else np.nan
    Hd = Hd * constants['kJ2J'] if Hd is not None else np.nan
    dS = dS * constants['kJ2J'] if dS is not None else np.nan

    if np.isnan(Ha):
        raise ValueError("Activation energy (Ha) has to be provided!")

    if np.isnan(Hd) and np.isnan(dS):
        param25 = param / np.exp(Ha * (Temp - Tref) / (Tref * constants['Rgas'] * Temp))
    elif not np.isnan(Hd) and not np.isnan(dS):
        param25 = param / (
            np.exp(Ha * (Temp - Tref) / (Tref * constants['Rgas'] * Temp)) *
            (1 + np.exp((Tref * dS - Hd) / (Tref * constants['Rgas']))) /
            (1 + np.exp((Temp * dS - Hd) / (Temp * constants['Rgas'])))
        )
    else:
        print("Warning: Both Hd and dS must be provided for full temperature response!")
        param25 = param / np.exp(Ha * (Temp - Tref) / (Tref * constants['Rgas'] * Temp))

    return param25

def stomatal_slope(data, Tair="Tair", pressure="pressure", GPP="GPP", Gs="Gs_mol",
                 VPD="VPD", Ca="Ca", Rleaf=None, model="USO", robust_nls=False,
                 nmin=40, fitg0=False, g0=0, fitD0=False, D0=1.5, Gamma=50,
                 missing_Rleaf_as_NA=False, constants=None):
    """
    Fit stomatal slope models (USO, Ball&Berry, Leuning).

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        Gs: Column name for stomatal conductance (mol m-2 s-1)
        VPD: Column name for vapor pressure deficit (kPa)
        Ca: Column name for atmospheric CO2 concentration (umol mol-1)
        Rleaf: Leaf respiration (umol CO2 m-2 s-1), optional
        model: Model type ('USO', 'Ball&Berry', 'Leuning')
        robust_nls: Boolean, use robust nonlinear least squares
        nmin: Minimum number of data points for fitting
        fitg0: Boolean, fit g0 parameter
        g0: Fixed g0 value if not fitting
        fitD0: Boolean, fit D0 parameter (Leuning model)
        D0: Fixed D0 value if not fitting (Leuning model)
        Gamma: CO2 compensation point (umol mol-1, Leuning model)
        missing_Rleaf_as_NA: If True, treat missing Rleaf as NA; else as 0
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Fitted model object (statsmodels or equivalent)
    """
    if constants is None:
        constants = bigleaf_constants()

    if model not in ["USO", "Ball&Berry", "Leuning"]:
        raise ValueError("model must be one of 'USO', 'Ball&Berry', 'Leuning'")

    check_input(data, [Tair, pressure, GPP, Gs, VPD, Ca])

    df = pd.DataFrame({
        'Tair': data[Tair], 'pressure': data[pressure], 'GPP': data[GPP],
        'Gs': data[Gs], 'VPD': data[VPD], 'Ca': data[Ca]
    })
    DwDc = constants['DwDc']

    if model == "Leuning":
        check_input(data, [Gamma])
        df['Gamma'] = Gamma

    if Rleaf is not None:
        if not missing_Rleaf_as_NA:
            Rleaf = np.where(np.isnan(Rleaf), 0, Rleaf)
    else:
        print("Respiration from the leaves is ignored and set to 0.")
        Rleaf = 0

    GPP = data[GPP] - Rleaf

    if model == "Leuning":
        nr_data = np.sum(~np.isnan(GPP) & ~np.isnan(df['Gs']) & ~np.isnan(df['VPD']) &
                        ~np.isnan(df['Ca']) & ~np.isnan(df['Gamma']))
    else:
        nr_data = np.sum(~np.isnan(GPP) & ~np.isnan(df['Gs']) & ~np.isnan(df['VPD']) &
                        ~np.isnan(df['Ca']))

    if nr_data < nmin:
        raise ValueError("Number of data points is less than nmin. g1 is not fitted.")

    # Note: Nonlinear least squares in Python is more complex than R's nls.
    # For simplicity, we'll use a placeholder for the model fitting, as statsmodels
    # doesn't directly support robust nonlinear regression like R's nlrob.
    # Users may need to implement this with scipy.optimize or another library.

    if model == "USO":
        def model_func(params, GPP, VPD, Ca, g0_fixed=None):
            g1 = params[0]
            g0 = params[1] if fitg0 else g0_fixed
            return g0 + DwDc * (1.0 + g1 / np.sqrt(VPD)) * GPP / Ca

        initial_guess = [3.0] if not fitg0 else [0.0, 3.0]
        # Placeholder for fitting logic
        print("Nonlinear fitting for USO model requires scipy.optimize implementation.")

    elif model == "Leuning":
        def model_func(params, GPP, Ca, VPD, Gamma, g0_fixed=None, D0_fixed=None):
            g1 = params[0]
            g0 = params[1] if fitg0 else g0_fixed
            D0 = params[2] if fitD0 else D0_fixed
            return g0 + g1 * GPP / ((Ca - Gamma) * (1 + VPD / D0))

        initial_guess = [9.0] if not (fitg0 or fitD0) else [0.0, 9.0, 1.5] if (fitg0 and fitD0) else [0.0, 9.0]
        # Placeholder for fitting logic
        print("Nonlinear fitting for Leuning model requires scipy.optimize implementation.")

    else:  # Ball&Berry
        rH = VPD_to_rH(df['VPD'], df['Tair'], constants=constants)
        df['rH'] = rH

        def model_func(params, GPP, rH, Ca, g0_fixed=None):
            g1 = params[0]
            g0 = params[1] if fitg0 else g0_fixed
            return g0 + g1 * (GPP * rH) / Ca

        initial_guess = [9.0] if not fitg0 else [0.0, 9.0]
        # Placeholder for fitting logic
        print("Nonlinear fitting for Ball&Berry model requires scipy.optimize implementation.")

    # Return a placeholder model object
    return None  # Replace with actual fitted model when implemented

def light_response(data, NEE="NEE", Reco="Reco", PPFD="PPFD", PPFD_ref=2000):
    """
    Fit light response curve for NEE.

    Args:
        data: DataFrame with required columns
        NEE: Column name for net ecosystem exchange (umol CO2 m-2 s-1)
        Reco: Column name for ecosystem respiration (umol CO2 m-2 s-1)
        PPFD: Column name for photosynthetic photon flux density (umol m-2 s-1)
        PPFD_ref: Reference PPFD (umol m-2 s-1)

    Returns:
        Fitted model object (placeholder)
    """
    check_input(data, [NEE, Reco, PPFD])

    def model_func(params, PPFD, Reco):
        alpha, GPP_ref = params
        return alpha * PPFD / (1 - (PPFD / PPFD_ref) + (alpha * PPFD / GPP_ref)) - Reco

    # Placeholder for fitting logic
    print("Nonlinear fitting for light response requires scipy.optimize implementation.")
    return None  # Replace with actual fitted model when implemented

def light_use_efficiency(GPP, PPFD):
    """
    Calculate light use efficiency.

    Args:
        GPP: Gross primary productivity (umol CO2 m-2 s-1)
        PPFD: Photosynthetic photon flux density (umol m-2 s-1)

    Returns:
        dict: Light use efficiency (LUE)
    """
    comp = np.isfinite(GPP) & np.isfinite(PPFD)
    LUE = np.sum(GPP[comp]) / np.sum(PPFD[comp])
    return {'LUE': LUE}

def stomatal_sensitivity(data, Gs="Gs_mol", VPD="VPD"):
    """
    Fit stomatal sensitivity to VPD.

    Args:
        data: DataFrame with required columns
        Gs: Column name for stomatal conductance (mol m-2 s-1)
        VPD: Column name for vapor pressure deficit (kPa)

    Returns:
        Fitted model object (placeholder)
    """
    check_input(data, [Gs, VPD])

    def model_func(params, VPD):
        m, b = params
        return -m * np.log(VPD) + b

    # Placeholder for fitting logic
    print("Nonlinear fitting for stomatal sensitivity requires scipy.optimize implementation.")
    return None  # Replace with actual fitted model when implemented

def decoupling(data, Tair="Tair", pressure="pressure", Ga="Ga_h", Gs="Gs_ms",
              approach="Jarvis&McNaughton_1986", LAI=None,
              Esat_formula="Sonntag_1990", constants=None):
    """
    Calculate decoupling coefficient (Omega).

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        Ga: Column name for aerodynamic conductance (m s-1)
        Gs: Column name for surface conductance (m s-1)
        approach: Decoupling approach ('Jarvis&McNaughton_1986' or 'Martin_1989')
        LAI: Leaf area index
        Esat_formula: Formula for saturation vapor pressure ('Sonntag_1990', 'Alduchov_1996', 'Allen_1998')
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Omega: Decoupling coefficient
    """
    if constants is None:
        constants = bigleaf_constants()

    if approach not in ["Jarvis&McNaughton_1986", "Martin_1989"]:
        raise ValueError("approach must be 'Jarvis&McNaughton_1986' or 'Martin_1989'")

    check_input(data, [Tair, pressure, Ga, Gs])

    Delta = Esat_slope(data[Tair], Esat_formula, constants)['Delta']
    gamma = psychrometric_constant(data[Tair], data[pressure], constants)
    epsilon = Delta / gamma

    if approach == "Jarvis&McNaughton_1986":
        Omega = (epsilon + 1) / (epsilon + 1 + data[Ga] / data[Gs])
    else:  # Martin_1989
        if LAI is None:
            raise ValueError("LAI is not provided!")
        Gr = longwave_conductance(data[Tair], LAI, constants)
        Omega = (epsilon + 1 + Gr / data[Ga]) / (epsilon + 1 + data[Ga] / data[Gs] + Gr / data[Gs] + Gr / data[Ga])

    return Omega

def longwave_conductance(Tair, LAI, constants=None):
    """
    Calculate longwave conductance.

    Args:
        Tair: Air temperature (degC)
        LAI: Leaf area index
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Gr: Longwave conductance (m s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair = Tair + constants['Kelvin']
    Gr = 4 * constants['sigma'] * Tair**3 * LAI / constants['cp']
    return Gr

def Esat_slope(Tair, formula="Sonntag_1990", constants=None):
    """
    Calculate saturation vapor pressure and its slope.

    Args:
        Tair: Air temperature (degC)
        formula: Formula for saturation vapor pressure ('Sonntag_1990', 'Alduchov_1996', 'Allen_1998')
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Esat (saturation vapor pressure, kPa), Delta (slope, kPa degC-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    if formula not in ["Sonntag_1990", "Alduchov_1996", "Allen_1998"]:
        raise ValueError("formula must be one of 'Sonntag_1990', 'Alduchov_1996', 'Allen_1998'")

    if formula == "Sonntag_1990":
        a, b, c = 611.2, 17.62, 243.12
    elif formula == "Alduchov_1996":
        a, b, c = 610.94, 17.625, 243.04
    else:  # Allen_1998
        a, b, c = 610.8, 17.27, 237.3

    Esat = a * np.exp((b * Tair) / (c + Tair)) * constants['Pa2kPa']
    Delta = a * (np.exp((b * Tair) / (c + Tair)) * (b / (c + Tair) - (b * Tair) / (c + Tair)**2)) * constants['Pa2kPa']

    return pd.DataFrame({'Esat': Esat, 'Delta': Delta})

def psychrometric_constant(Tair, pressure, constants=None):
    """
    Calculate psychrometric constant.

    Args:
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        gamma: Psychrometric constant (kPa degC-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    lambda_ = latent_heat_vaporization(Tair)
    gamma = (constants['cp'] * pressure) / (constants['eps'] * lambda_)
    return gamma

def VPD_to_rH(VPD, Tair, Esat_formula="Sonntag_1990", constants=None):
    """
    Convert VPD to relative humidity.

    Args:
        VPD: Vapor pressure deficit (kPa)
        Tair: Air temperature (degC)
        Esat_formula: Formula for saturation vapor pressure ('Sonntag_1990', 'Alduchov_1996', 'Allen_1998')
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        rH: Relative humidity (fraction)
    """
    if constants is None:
        constants = bigleaf_constants()

    esat = Esat_slope(Tair, Esat_formula, constants)['Esat']
    rH = 1 - VPD / esat
    return rH

### bigleaf-p3

In [19]:
import numpy as np
import pandas as pd
from datetime import datetime

def biochemical_energy(data, GPP="GPP", Reco="Reco", constants=None):
    """
    Calculate biochemical energy from GPP and respiration.

    Args:
        data: DataFrame with required columns
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        Reco: Column name for ecosystem respiration (umol CO2 m-2 s-1)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Energy from GPP, Reco, and NEE (W m-2)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [GPP, Reco])

    GPP_energy = data[GPP] * 0.219 * constants['umol2mol'] * constants['kg2g']
    Reco_energy = data[Reco] * 0.219 * constants['umol2mol'] * constants['kg2g']
    NEE_energy = (data[GPP] - data[Reco]) * 0.219 * constants['umol2mol'] * constants['kg2g']

    return pd.DataFrame({
        'GPP_energy': GPP_energy,
        'Reco_energy': Reco_energy,
        'NEE_energy': NEE_energy
    })

def energy_use_efficiency(data, Rn="Rn", GPP="GPP", Reco="Reco", constants=None):
    """
    Calculate energy use efficiency (EUE).

    Args:
        data: DataFrame with required columns
        Rn: Column name for net radiation (W m-2)
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        Reco: Column name for ecosystem respiration (umol CO2 m-2 s-1)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        dict: EUE (fraction)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Rn, GPP, Reco])

    biochemical = biochemical_energy(data, GPP, Reco, constants)
    EUE = np.nanmedian(biochemical['GPP_energy'] / data[Rn])

    return {'EUE': EUE}

def energy_closure(data, Rn="Rn", LE="LE", H="H", G=None, S=None):
    """
    Calculate energy balance closure.

    Args:
        data: DataFrame with required columns
        Rn: Column name for net radiation (W m-2)
        LE: Column name for latent heat flux (W m-2)
        H: Column name for sensible heat flux (W m-2)
        G: Column name for ground heat flux (W m-2, optional)
        S: Column name for storage flux (W m-2, optional)

    Returns:
        dict: Energy balance closure metrics
    """
    check_input(data, [Rn, LE, H] + ([G] if G is not None else []) + ([S] if S is not None else []))

    A = data[Rn] - (data[LE] + data[H] + (data[G] if G is not None else 0) + (data[S] if S is not None else 0))
    EBR = np.nanmean(data[LE] + data[H] + (data[G] if G is not None else 0) + (data[S] if S is not None else 0)) / np.nanmean(data[Rn])
    RMSE = np.sqrt(np.nanmean(A**2))

    return {'EBR': EBR, 'RMSE': RMSE, 'A': A}

def isothermal_net_radiation(data, Tair="Tair", Rn="Rn", albedo=0.23, constants=None):
    """
    Calculate isothermal net radiation.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        Rn: Column name for net radiation (W m-2)
        albedo: Surface albedo (fraction)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Isothermal net radiation components
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, Rn])

    Rabs = data[Rn] / (1 - albedo)
    Tair_K = data[Tair] + constants['Kelvin']
    Rnl_iso = -4 * constants['sigma'] * Tair_K**3 * (data[Tair] - data[Tair])
    Rn_iso = (1 - albedo) * Rabs + Rnl_iso

    return pd.DataFrame({'Rabs': Rabs, 'Rnl_iso': Rnl_iso, 'Rn_iso': Rn_iso})

def potential_ET(data, Tair="Tair", pressure="pressure", Rn="Rn", G=None, S=None,
                missing_G_as_NA=False, missing_S_as_NA=False,
                Esat_formula="Sonntag_1990", constants=None):
    """
    Calculate potential evapotranspiration.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        Rn: Column name for net radiation (W m-2)
        G: Column name for ground heat flux (W m-2, optional)
        S: Column name for storage flux (W m-2, optional)
        missing_G_as_NA: If True, treat missing G as NA; else as 0
        missing_S_as_NA: If True, treat missing S as NA; else as 0
        Esat_formula: Formula for saturation vapor pressure
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Potential ET (mm s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, pressure, Rn] + ([G] if G is not None else []) + ([S] if S is not None else []))

    if G is not None and not missing_G_as_NA:
        G = np.where(np.isnan(data[G]), 0, data[G])
    else:
        G = 0
    if S is not None and not missing_S_as_NA:
        S = np.where(np.isnan(data[S]), 0, data[S])
    else:
        S = 0

    Delta = Esat_slope(data[Tair], Esat_formula, constants)['Delta']
    gamma = psychrometric_constant(data[Tair], data[pressure], constants)
    lambda_ = latent_heat_vaporization(data[Tair])

    ETpot = (Delta * (data[Rn] - G - S)) / (Delta + gamma) / lambda_
    ETpot = np.maximum(ETpot, 0)

    return pd.DataFrame({'ETpot': ETpot})

def equilibrium_imposed_ET(data, Tair="Tair", pressure="pressure", Rn="Rn", G=None, S=None,
                          VPD="VPD", Ga="Ga_h", missing_G_as_NA=False, missing_S_as_NA=False,
                          Esat_formula="Sonntag_1990", constants=None):
    """
    Calculate equilibrium and imposed evapotranspiration.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        Rn: Column name for net radiation (W m-2)
        G: Column name for ground heat flux (W m-2, optional)
        S: Column name for storage flux (W m-2, optional)
        VPD: Column name for vapor pressure deficit (kPa)
        Ga: Column name for aerodynamic conductance (m s-1)
        missing_G_as_NA: If True, treat missing G as NA; else as 0
        missing_S_as_NA: If True, treat missing S as NA; else as 0
        Esat_formula: Formula for saturation vapor pressure
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Equilibrium and imposed ET (mm s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, pressure, Rn, VPD, Ga] + ([G] if G is not None else []) + ([S] if S is not None else []))

    if G is not None and not missing_G_as_NA:
        G = np.where(np.isnan(data[G]), 0, data[G])
    else:
        G = 0
    if S is not None and not missing_S_as_NA:
        S = np.where(np.isnan(data[S]), 0, data[S])
    else:
        S = 0

    Delta = Esat_slope(data[Tair], Esat_formula, constants)['Delta']
    gamma = psychrometric_constant(data[Tair], data[pressure], constants)
    lambda_ = latent_heat_vaporization(data[Tair])
    rho = air_density(data[Tair], data[pressure], constants)

    ETequ = (Delta * (data[Rn] - G - S)) / (Delta + gamma) / lambda_
    ETequ = np.maximum(ETequ, 0)
    ETimp = (rho * constants['cp'] * data[VPD] * data[Ga]) / (gamma * lambda_)

    return pd.DataFrame({'ETequ': ETequ, 'ETimp': ETimp})

def reference_ET(data, Tair="Tair", pressure="pressure", Rn="Rn", G=None, VPD="VPD",
                ustar="ustar", zr=40, zh=25, d=16.75, z0m=2,
                missing_G_as_NA=False, Esat_formula="Sonntag_1990", constants=None):
    """
    Calculate reference evapotranspiration.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        Rn: Column name for net radiation (W m-2)
        G: Column name for ground heat flux (W m-2, optional)
        VPD: Column name for vapor pressure deficit (kPa)
        ustar: Column name for friction velocity (m s-1)
        zr: Reference height (m)
        zh: Canopy height (m)
        d: Displacement height (m)
        z0m: Roughness length for momentum (m)
        missing_G_as_NA: If True, treat missing G as NA; else as 0
        Esat_formula: Formula for saturation vapor pressure
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Reference ET (mm s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, pressure, Rn, VPD, ustar] + ([G] if G is not None else []))

    if G is not None and not missing_G_as_NA:
        G = np.where(np.isnan(data[G]), 0, data[G])
    else:
        G = 0

    Delta = Esat_slope(data[Tair], Esat_formula, constants)['Delta']
    gamma = psychrometric_constant(data[Tair], data[pressure], constants)
    lambda_ = latent_heat_vaporization(data[Tair])
    rho = air_density(data[Tair], data[pressure], constants)

    kB_h = 2
    Rb_h = kB_h / (constants['k'] * data[ustar])
    Ra_m = np.maximum(np.log((zr - d) / z0m), 0) / (constants['k'] * data[ustar])
    Ra_h = Ra_m + Rb_h
    Ga_h = 1 / Ra_h

    ETo = (Delta * (data[Rn] - G) + (rho * constants['cp'] * data[VPD] * Ga_h)) / (Delta + gamma) / lambda_
    ETo = np.maximum(ETo, 0)

    return pd.DataFrame({'ETo': ETo})

def filter_data(data, filter_growseas=False, filter_precip=False, filter_ustar=False,
                filter_swdev=False, filter_vars=None, nmin=30, ustar_threshold=0.2,
                ustar_percentile=0.2, precip_threshold=0.5, sw_threshold=0.1,
                ws_threshold=10, growing_season=True, growing_Tair_threshold=10,
                growing_precip_threshold=0.1, growing_days=5, constants=None):
    """
    Filter data based on quality control and environmental conditions.

    Args:
        data: DataFrame with required columns
        filter_growseas: Boolean, filter for growing season
        filter_precip: Boolean, filter for precipitation
        filter_ustar: Boolean, filter for friction velocity
        filter_swdev: Boolean, filter for soil water deviation
        filter_vars: List of variables to check for NA
        nmin: Minimum number of data points
        ustar_threshold: Friction velocity threshold (m s-1)
        ustar_percentile: Percentile for ustar threshold
        precip_threshold: Precipitation threshold (mm)
        sw_threshold: Soil water deviation threshold
        ws_threshold: Wind speed threshold (m s-1)
        growing_season: Boolean, filter for growing season
        growing_Tair_threshold: Temperature threshold for growing season (degC)
        growing_precip_threshold: Precipitation threshold for growing season (mm)
        growing_days: Number of days for growing season filter
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Filtered data
    """
    if constants is None:
        constants = bigleaf_constants()

    filtered_data = data.copy()

    if filter_vars is not None:
        for var in filter_vars:
            filtered_data = filtered_data[~filtered_data[var].isna()]

    if filter_ustar:
        filtered_data = filtered_data[filtered_data['ustar'] >= ustar_threshold]

    if filter_precip:
        filtered_data = filtered_data[filtered_data['precip'] < precip_threshold]

    if filter_swdev:
        filtered_data = filtered_data[filtered_data['sw_dev'] < sw_threshold]

    if filter_growseas and growing_season:
        growing = (filtered_data['Tair'] >= growing_Tair_threshold) & (filtered_data['precip'] >= growing_precip_threshold)
        filtered_data = filtered_data[growing]

    if len(filtered_data) < nmin:
        print("Warning: Number of data points after filtering is less than nmin.")

    return filtered_data

def air_density(Tair, pressure, constants=None):
    """
    Calculate air density.

    Args:
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        rho: Air density (kg m-3)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    rho = (pressure * constants['kPa2Pa']) / (constants['Rd'] * Tair_K)
    return rho

def pressure_from_elevation(elevation, Tair=15, constants=None):
    """
    Calculate atmospheric pressure from elevation.

    Args:
        elevation: Elevation above sea level (m)
        Tair: Air temperature (degC)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        pressure: Atmospheric pressure (kPa)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    pressure = constants['pressure0'] * np.exp(-constants['g'] * elevation / (constants['Rd'] * Tair_K))
    return pressure * constants['Pa2kPa']

def kinematic_viscosity(Tair, pressure, constants=None):
    """
    Calculate kinematic viscosity of air.

    Args:
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        v: Kinematic viscosity (m2 s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    v = (1.81e-5 * (Tair_K / 293.15)**1.5 * (293.15 + 110.4) / (Tair_K + 110.4)) / air_density(Tair, pressure, constants)
    return v

def Reynolds_Number(Tair, pressure, ustar, hs, constants=None):
    """
    Calculate Reynolds number.

    Args:
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        ustar: Friction velocity (m s-1)
        hs: Soil roughness length (m)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Re: Reynolds number
    """
    if constants is None:
        constants = bigleaf_constants()

    v = kinematic_viscosity(Tair, pressure, constants)
    Re = ustar * hs / v
    return Re

def optimum_temperature(data, GPP="GPP", Tair="Tair", nmin=30, Tmin=0, Tmax=40,
                       Tinterval=0.2, perc_low=0.05, perc_high=0.95, min_data=10):
    """
    Determine optimum temperature for GPP.

    Args:
        data: DataFrame with required columns
        GPP: Column name for gross primary productivity (umol CO2 m-2 s-1)
        Tair: Column name for air temperature (degC)
        nmin: Minimum number of data points
        Tmin: Minimum temperature (degC)
        Tmax: Maximum temperature (degC)
        Tinterval: Temperature interval (degC)
        perc_low: Lower percentile for boundary line
        perc_high: Upper percentile for boundary line
        min_data: Minimum data points per bin

    Returns:
        dict: Optimum temperature metrics
    """
    check_input(data, [GPP, Tair])

    data = data[(data[Tair] >= Tmin) & (data[Tair] <= Tmax)]
    if len(data) < nmin:
        print("Warning: Number of data points is less than nmin.")
        return {'Topt': np.nan, 'Topt_SE': np.nan}

    Tbins = np.arange(Tmin, Tmax + Tinterval, Tinterval)
    GPP_max = [np.percentile(data[GPP][(data[Tair] >= Tbins[i]) & (data[Tair] < Tbins[i+1])], perc_high)
               if np.sum((data[Tair] >= Tbins[i]) & (data[Tair] < Tbins[i+1])) >= min_data else np.nan
               for i in range(len(Tbins)-1)]

    Topt = Tbins[np.argmax(GPP_max)]
    Topt_SE = np.nanstd([Tbins[i] for i in range(len(GPP_max)) if np.isfinite(GPP_max[i])]) / np.sqrt(np.sum(np.isfinite(GPP_max)))

    return {'Topt': Topt, 'Topt_SE': Topt_SE}

def potential_radiation(timestamp, lat, lon, elevation=0, timezone=0, method="M spitting", constants=None):
    """
    Calculate potential and extraterrestrial radiation.

    Args:
        timestamp: Array of datetime objects
        lat: Latitude (degrees)
        lon: Longitude (degrees)
        elevation: Elevation above sea level (m)
        timezone: Timezone offset (hours)
        method: Radiation calculation method ('M spitting' or 'Duffie')
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Extraterrestrial and potential radiation (W m-2)
    """
    if constants is None:
        constants = bigleaf_constants()

    if method not in ["M spitting", "Duffie"]:
        raise ValueError("method must be 'M spitting' or 'Duffie'")

    # Simplified solar position calculation (placeholder)
    jday = np.array([t.timetuple().tm_yday for t in timestamp])
    hour = np.array([t.hour + t.minute / 60 + timezone for t in timestamp])

    lat_rad = np.deg2rad(lat)
    delta = 0.409 * np.sin(2 * np.pi * (jday - 81) / 365)
    cos_zenith = np.sin(lat_rad) * np.sin(delta) + np.cos(lat_rad) * np.cos(delta) * np.cos(2 * np.pi * (hour - 12) / 24)
    cos_zenith = np.maximum(cos_zenith, 0)

    Rext = constants['solar_constant'] * (1 + 0.033 * np.cos(2 * np.pi * jday / 365)) * cos_zenith
    Rpot = Rext * 0.75  # Simplified atmospheric transmission

    return pd.DataFrame({'Rext': Rext, 'Rpot': Rpot})

def stability_parameter(data, Tair="Tair", pressure="pressure", ustar="ustar", H="H", zr=40, d=16.75, constants=None):
    """
    Calculate Monin-Obukhov stability parameter.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        ustar: Column name for friction velocity (m s-1)
        H: Column name for sensible heat flux (W m-2)
        zr: Reference height (m)
        d: Displacement height (m)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        zeta: Stability parameter
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, pressure, ustar, H])

    Tair_K = data[Tair] + constants['Kelvin']
    rho = air_density(data[Tair], data[pressure], constants)
    L = -rho * constants['cp'] * Tair_K * data[ustar]**3 / (constants['k'] * constants['g'] * data[H])
    zeta = (zr - d) / L
    return zeta

def stability_correction(zeta, formulation="Dyer_1970"):
    """
    Calculate stability corrections for momentum and heat.

    Args:
        zeta: Stability parameter
        formulation: Stability formulation ('Dyer_1970' or 'Businger_1971')

    Returns:
        DataFrame: Stability corrections (psi_m, psi_h)
    """
    if formulation not in ["Dyer_1970", "Businger_1971"]:
        raise ValueError("formulation must be 'Dyer_1970' or 'Businger_1971'")

    psi_m = psi_h = np.zeros_like(zeta)

    stable = zeta > 0
    unstable = zeta < 0

    if formulation == "Dyer_1970":
        psi_m[stable] = -5 * zeta[stable]
        psi_h[stable] = -5 * zeta[stable]
        x = (1 - 16 * zeta[unstable])**0.25
        psi_m[unstable] = 2 * np.log((1 + x) / 2) + np.log((1 + x**2) / 2) - 2 * np.arctan(x) + np.pi / 2
        psi_h[unstable] = 2 * np.log((1 + x**2) / 2)
    else:  # Businger_1971
        psi_m[stable] = -4.7 * zeta[stable]
        psi_h[stable] = -4.7 * zeta[stable]
        x = (1 - 15 * zeta[unstable])**0.25
        psi_m[unstable] = 2 * np.log((1 + x) / 2) + np.log((1 + x**2) / 2) - 2 * np.arctan(x) + np.pi / 2
        psi_h[unstable] = 2 * np.log((1 + x**2) / 2)

    return pd.DataFrame({'psi_m': psi_m, 'psi_h': psi_h})

def surface_conditions(data, Tair="Tair", pressure="pressure", Rn="Rn", LE="LE", H="H",
                      Ga="Ga_h", albedo=0.23, emissivity=0.96, Esat_formula="Sonntag_1990", constants=None):
    """
    Calculate surface temperature, humidity, and CO2 concentration.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        Rn: Column name for net radiation (W m-2)
        LE: Column name for latent heat flux (W m-2)
        H: Column name for sensible heat flux (W m-2)
        Ga: Column name for aerodynamic conductance (m s-1)
        albedo: Surface albedo (fraction)
        emissivity: Surface emissivity
        Esat_formula: Formula for saturation vapor pressure
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Surface conditions (Ts, es, qs, Cs)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, pressure, Rn, LE, H, Ga])

    Tair_K = data[Tair] + constants['Kelvin']
    rho = air_density(data[Tair], data[pressure], constants)
    Ts = Tair_K + data[H] / (rho * constants['cp'] * data[Ga])

    esat = Esat_slope(Ts - constants['Kelvin'], Esat_formula, constants)['Esat']
    qs = constants['eps'] * esat / (data[pressure] - (1 - constants['eps']) * esat)

    return pd.DataFrame({'Ts': Ts - constants['Kelvin'], 'es': esat, 'qs': qs})

def surface_conductance(data, Tair="Tair", pressure="pressure", VPD="VPD", LE="LE", Ga="Ga_h",
                       formulation="Penman_Monteith", Esat_formula="Sonntag_1990", constants=None):
    """
    Calculate surface conductance.

    Args:
        data: DataFrame with required columns
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        VPD: Column name for vapor pressure deficit (kPa)
        LE: Column name for latent heat flux (W m-2)
        Ga: Column name for aerodynamic conductance (m s-1)
        formulation: Conductance formulation ('Penman_Monteith' or 'Flux_Gradient')
        Esat_formula: Formula for saturation vapor pressure
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        DataFrame: Surface conductance (Gs_ms, Gs_mol)
    """
    if constants is None:
        constants = bigleaf_constants()

    if formulation not in ["Penman_Monteith", "Flux_Gradient"]:
        raise ValueError("formulation must be 'Penman_Monteith' or 'Flux_Gradient'")

    check_input(data, [Tair, pressure, VPD, LE, Ga])

    Delta = Esat_slope(data[Tair], Esat_formula, constants)['Delta']
    gamma = psychrometric_constant(data[Tair], data[pressure], constants)
    lambda_ = latent_heat_vaporization(data[Tair])
    rho = air_density(data[Tair], data[pressure], constants)

    if formulation == "Penman_Monteith":
        Gs_ms = (data[Ga] * data[LE] * gamma) / (Delta * data[LE] + rho * constants['cp'] * data[VPD] * data[Ga] - data[LE] * gamma)
    else:  # Flux_Gradient
        ET = LE_to_ET(data[LE], data[Tair])
        Gs_ms = ET / (data[VPD] / data[pressure])

    Tair_K = data[Tair] + constants['Kelvin']
    Gs_mol = Gs_ms * data[pressure] * constants['kPa2Pa'] / (constants['Rgas'] * Tair_K)

    return pd.DataFrame({'Gs_ms': Gs_ms, 'Gs_mol': Gs_mol})

def roughness_parameters(method="wind_profile", zh=25, zr=40, d=16.75, z0m=2, data=None,
                        Tair="Tair", pressure="pressure", wind="wind", ustar="ustar", H="H",
                        stab_roughness=True, stab_formulation="Dyer_1970", constants=None):
    """
    Estimate roughness parameters.

    Args:
        method: Estimation method ('wind_profile' or 'simple')
        zh: Canopy height (m)
        zr: Reference height (m)
        d: Displacement height (m)
        z0m: Roughness length for momentum (m)
        data: DataFrame with required columns (if method='wind_profile')
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        wind: Column name for wind speed (m s-1)
        ustar: Column name for friction velocity (m s-1)
        H: Column name for sensible heat flux (W m-2)
        stab_roughness: Boolean, apply stability correction
        stab_formulation: Stability formulation ('Dyer_1970' or 'Businger_1971')
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        dict: Roughness parameters (d, z0m)
    """
    if constants is None:
        constants = bigleaf_constants()

    if method not in ["wind_profile", "simple"]:
        raise ValueError("method must be 'wind_profile' or 'simple'")

    if method == "wind_profile":
        check_input(data, [Tair, pressure, wind, ustar, H])

        if stab_roughness:
            zeta = stability_parameter(data, Tair, pressure, ustar, H, zr, d, constants)
            psi_m = stability_correction(zeta, stab_formulation)['psi_m']
            z0m = (zr - d) / np.exp(data[wind] / data[ustar] * constants['k'] + psi_m)
        else:
            z0m = (zr - d) / np.exp(data[wind] / data[ustar] * constants['k'])
        d = 0.67 * zh  # Simplified estimate
    else:  # simple
        d = 0.67 * zh
        z0m = 0.1 * zh

    return {'d': d, 'z0m': z0m}

def wind_profile(data, z, Tair="Tair", pressure="pressure", ustar="ustar", H="H", wind="wind",
                zr=40, zh=25, d=16.75, z0m=2, estimate_z0m=True, stab_correction=True,
                stab_formulation="Dyer_1970", constants=None):
    """
    Calculate wind speed at height z.

    Args:
        data: DataFrame with required columns
        z: Height for wind speed calculation (m)
        Tair: Column name for air temperature (degC)
        pressure: Column name for atmospheric pressure (kPa)
        ustar: Column name for friction velocity (m s-1)
        H: Column name for sensible heat flux (W m-2)
        wind: Column name for wind speed (m s-1)
        zr: Reference height (m)
        zh: Canopy height (m)
        d: Displacement height (m)
        z0m: Roughness length for momentum (m)
        estimate_z0m: Boolean, estimate z0m from wind profile
        stab_correction: Boolean, apply stability correction
        stab_formulation: Stability formulation ('Dyer_1970' or 'Businger_1971')
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        wind_z: Wind speed at height z (m s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    check_input(data, [Tair, pressure, ustar, H, wind])

    if estimate_z0m:
        roughness = roughness_parameters("wind_profile", zh, zr, d, z0m, data, Tair, pressure, wind, ustar, H,
                                        stab_roughness=stab_correction, stab_formulation=stab_formulation, constants=constants)
        z0m = roughness['z0m']

    if stab_correction:
        zeta = stability_parameter(data, Tair, pressure, ustar, H, zr, d, constants)
        psi_m = stability_correction(zeta, stab_formulation)['psi_m']
        wind_z = data[ustar] / constants['k'] * (np.log((z - d) / z0m) - psi_m)
    else:
        wind_z = data[ustar] / constants['k'] * np.log((z - d) / z0m)

    return wind_z

def ET_to_LE(ET, Tair):
    """
    Convert evapotranspiration to latent heat flux.

    Args:
        ET: Evapotranspiration (kg H2O m-2 s-1)
        Tair: Air temperature (degC)

    Returns:
        LE: Latent heat flux (W m-2)
    """
    lambda_ = latent_heat_vaporization(Tair)
    LE = ET * lambda_
    return LE

def ms_to_mol(Gs_ms, Tair, pressure, constants=None):
    """
    Convert conductance from m s-1 to mol m-2 s-1.

    Args:
        Gs_ms: Conductance (m s-1)
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Gs_mol: Conductance (mol m-2 s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    Gs_mol = Gs_ms * (pressure * constants['kPa2Pa']) / (constants['Rgas'] * Tair_K)
    return Gs_mol

def mol_to_ms(Gs_mol, Tair, pressure, constants=None):
    """
    Convert conductance from mol m-2 s-1 to m s-1.

    Args:
        Gs_mol: Conductance (mol m-2 s-1)
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        Gs_ms: Conductance (m s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    Gs_ms = Gs_mol * (constants['Rgas'] * Tair_K) / (pressure * constants['kPa2Pa'])
    return Gs_ms

def flux_to_ustar(flux, Tair, pressure, wind, constants=None):
    """
    Convert momentum flux to friction velocity.

    Args:
        flux: Momentum flux (kg m-1 s-2)
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        wind: Wind speed (m s-1)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        ustar: Friction velocity (m s-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    rho = air_density(Tair, pressure, constants)
    ustar = np.sqrt(flux / rho)
    return ustar

def CO2_to_umol(CO2, Tair, pressure, constants=None):
    """
    Convert CO2 concentration to micromoles per cubic meter.

    Args:
        CO2: CO2 concentration (umol mol-1)
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        CO2_umol: CO2 concentration (umol m-3)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    CO2_umol = CO2 * (pressure * constants['kPa2Pa']) / (constants['Rgas'] * Tair_K)
    return CO2_umol

def umol_to_CO2(CO2_umol, Tair, pressure, constants=None):
    """
    Convert CO2 concentration from micromoles per cubic meter to umol mol-1.

    Args:
        CO2_umol: CO2 concentration (umol m-3)
        Tair: Air temperature (degC)
        pressure: Atmospheric pressure (kPa)
        constants: Dictionary of constants from bigleaf_constants()

    Returns:
        CO2: CO2 concentration (umol mol-1)
    """
    if constants is None:
        constants = bigleaf_constants()

    Tair_K = Tair + constants['Kelvin']
    CO2 = CO2_umol * (constants['Rgas'] * Tair_K) / (pressure * constants['kPa2Pa'])
    return CO2

def Rg_to_PPFD(Rg, J_to_mol=4.6, frac_PAR=0.5):
    """
    Convert global radiation to photosynthetic photon flux density (PPFD).

    Args:
        Rg: Global radiation (W m-2) as scalar, NumPy array, pandas Series, or DataFrame column
        J_to_mol: Conversion factor from energy to photons (umol J-1, default 4.6)
        frac_PAR: Fraction of global radiation that is photosynthetically active (default 0.5)

    Returns:
        PPFD: Photosynthetic photon flux density (umol m-2 s-1)
    """
    # Validate input
    vars_dict = check_input(None, [Rg])
    Rg = vars_dict[Rg if isinstance(Rg, str) else Rg.name if isinstance(Rg, pd.Series) else 'Rg']

    # Calculate PPFD
    PPFD = Rg * frac_PAR * J_to_mol

    return PPFD

def PPFD_to_Rg(PPFD, J_to_mol=4.6, frac_PAR=0.5):
    """
    Convert photosynthetic photon flux density (PPFD) to global radiation.

    Args:
        PPFD: Photosynthetic photon flux density (umol m-2 s-1) as scalar, NumPy array, pandas Series, or DataFrame column
        J_to_mol: Conversion factor from energy to photons (umol J-1, default 4.6)
        frac_PAR: Fraction of global radiation that is photosynthetically active (default 0.5)

    Returns:
        Rg: Global radiation (W m-2)
    """
    # Validate input
    vars_dict = check_input(None, [PPFD])
    PPFD = vars_dict[PPFD if isinstance(PPFD, str) else PPFD.name if isinstance(PPFD, pd.Series) else 'PPFD']

    # Calculate Rg
    Rg = PPFD / frac_PAR / J_to_mol

    return Rg

,PPFD
0,4.347826
1,4.782609


### Run test

In [6]:
# Part 1 (lines 1–411): Covered WUE_metrics.r, aerodynamic_conductance.r, and bigleaf_constants.r.
# Part 2 (lines 412–822): Covered bigleaf_physiology.r, boundary_layer_conductance.r, check_input.r, and decoupling.r.
# Part 3 (lines 823–1233): Covered energy_balance.r, evapotranspiration.r, filter_data.r, meteorological_variables.r, optimum_temperature.r, potential_radiation.r, stability_correction.r, surface_conditions.r, surface_conductance.r, surface_roughness.r, and unit_conversions.r.

In [7]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def generate_dummy_data(n_timesteps=48, start_date="2023-07-01"):
    """
    Generate dummy micrometeorological data for testing bigleaf functions.

    Args:
        n_timesteps: Number of timesteps (default: 48, half-hourly for 1 day)
        start_date: Start date for timestamp (format: 'YYYY-MM-DD')

    Returns:
        pd.DataFrame: Dummy dataset with required variables
    """
    # Time settings: half-hourly timesteps
    timestamps = [datetime.strptime(start_date, "%Y-%m-%d") + timedelta(minutes=30 * i) for i in range(n_timesteps)]
    hours = np.array([t.hour + t.minute / 60 for t in timestamps])

    # Simulate diurnal cycle using a cosine function for realism
    diurnal = np.cos(2 * np.pi * (hours - 12) / 24)

    # Initialize DataFrame
    data = pd.DataFrame({'timestamp': timestamps})

    # Core meteorological variables
    data['Tair'] = 20 + 5 * diurnal  # Air temperature (degC), 15–25°C
    data['pressure'] = 101.3 - 0.1 * np.random.randn(n_timesteps)  # Atmospheric pressure (kPa)
    data['VPD'] = 1.5 + 0.5 * diurnal + 0.1 * np.random.randn(n_timesteps)  # Vapor pressure deficit (kPa), 1–2 kPa
    data['wind'] = 2 + 1 * np.abs(diurnal) + 0.2 * np.random.randn(n_timesteps)  # Wind speed (m s-1), 1–3 m s-1
    data['ustar'] = 0.3 + 0.1 * np.abs(diurnal) + 0.05 * np.random.randn(n_timesteps)  # Friction velocity (m s-1), 0.2–0.4 m s-1

    # Flux variables
    data['GPP'] = np.maximum(0, 10 * diurnal + 2 * np.random.randn(n_timesteps))  # Gross primary productivity (umol CO2 m-2 s-1), 0–12
    data['NEE'] = -data['GPP'] + 2 + 0.5 * np.random.randn(n_timesteps)  # Net ecosystem exchange (umol CO2 m-2 s-1), -10 to 4
    data['Reco'] = 2 + 0.5 * np.abs(diurnal) + 0.2 * np.random.randn(n_timesteps)  # Ecosystem respiration (umol CO2 m-2 s-1), 1–3
    data['LE'] = np.maximum(0, 100 * diurnal + 20 * np.random.randn(n_timesteps))  # Latent heat flux (W m-2), 0–120
    data['H'] = np.maximum(0, 80 * diurnal + 15 * np.random.randn(n_timesteps))  # Sensible heat flux (W m-2), 0–100
    data['Rn'] = np.maximum(0, 200 * diurnal + 30 * np.random.randn(n_timesteps))  # Net radiation (W m-2), 0–230
    data['G'] = 10 + 5 * diurnal + 2 * np.random.randn(n_timesteps)  # Ground heat flux (W m-2), 5–15
    data['S'] = 5 + 2 * diurnal + 1 * np.random.randn(n_timesteps)  # Storage flux (W m-2), 3–7

    # Conductance variables
    data['Ga_h'] = 0.05 + 0.02 * np.abs(diurnal) + 0.005 * np.random.randn(n_timesteps)  # Aerodynamic conductance (m s-1), 0.03–0.07
    data['Gs_ms'] = 0.01 + 0.005 * diurnal + 0.001 * np.random.randn(n_timesteps)  # Surface conductance (m s-1), 0.005–0.015
    data['Gs_mol'] = 0.4 + 0.2 * diurnal + 0.05 * np.random.randn(n_timesteps)  # Surface conductance (mol m-2 s-1), 0.2–0.6

    # Photosynthetic and CO2 variables
    data['PPFD'] = np.maximum(0, 1000 * diurnal + 100 * np.random.randn(n_timesteps))  # Photosynthetic photon flux density (umol m-2 s-1), 0–1200
    data['Ca'] = 400 + 10 * np.random.randn(n_timesteps)  # Atmospheric CO2 concentration (umol mol-1), 390–410
    data['Ci'] = 300 + 20 * diurnal + 10 * np.random.randn(n_timesteps)  # Intercellular CO2 concentration (umol mol-1), 280–320

    # Precipitation and soil variables
    data['precip'] = np.where(np.random.rand(n_timesteps) < 0.1, 0.5 + 0.2 * np.random.rand(n_timesteps), 0)  # Precipitation (mm), mostly 0, occasional 0.5–0.7
    data['sw_dev'] = 0.05 + 0.02 * np.random.randn(n_timesteps)  # Soil water deviation (fraction), 0.03–0.07

    # Location and site characteristics
    data['lat'] = 45.0  # Latitude (degrees)
    data['lon'] = -75.0  # Longitude (degrees)
    data['elevation'] = 100  # Elevation (m)

    return data

# Example usage
if __name__ == "__main__":
    dummy_data = generate_dummy_data()
    print(dummy_data.head())
    dummy_data.to_csv("dummy_bigleaf_data.csv", index=False)

            timestamp       Tair    pressure       VPD      wind     ustar  \
0 2023-07-01 00:00:00  15.000000  101.207170  0.917708  2.658674  0.461747   
1 2023-07-01 00:30:00  15.042776  101.143237  1.061694  3.183455  0.378072   
2 2023-07-01 01:00:00  15.170371  101.202035  1.034346  3.162047  0.380193   
3 2023-07-01 01:30:00  15.380602  101.376046  1.141937  3.015341  0.307330   
4 2023-07-01 02:00:00  15.669873  101.246969  0.705228  2.786998  0.433037   

   GPP       NEE      Reco   LE  ...     Gs_ms    Gs_mol  PPFD          Ca  \
0  0.0  0.893396  2.180154  0.0  ...  0.005837  0.182843   0.0  405.800835   
1  0.0  0.893510  2.620667  0.0  ...  0.003595  0.150492   0.0  388.703339   
2  0.0  2.538097  2.527734  0.0  ...  0.005251  0.227829   0.0  405.843448   
3  0.0  2.290616  2.472232  0.0  ...  0.006399  0.126683   0.0  409.135857   
4  0.0  1.915650  2.409719  0.0  ...  0.005335  0.212088   0.0  392.510846   

           Ci  precip    sw_dev   lat   lon  elevation  
0  27

In [8]:
import warnings

warnings.simplefilter('ignore')

In [9]:
# generate_dummy_data().columns

In [9]:
import pandas as pd
# from bigleaf.bigleaf_part1 import (
#     WUE_metrics, Gb_Thom, Gb_Choudhury, Gb_Su, aerodynamic_conductance, bigleaf_constants,
#     roughness_length_heat
# )
# from bigleaf.bigleaf_part2 import (
#     intercellular_CO2, photosynthetic_capacity, Arrhenius_temp_response, decoupling,
#     longwave_conductance, Esat_slope, psychrometric_constant, VPD_to_rH
# )
# from bigleaf.bigleaf_part3 import (
#     biochemical_energy, energy_use_efficiency, energy_closure, isothermal_net_radiation,
#     potential_ET, equilibrium_imposed_ET, reference_ET, filter_data, air_density,
#     pressure_from_elevation, kinematic_viscosity, Reynolds_Number, optimum_temperature,
#     potential_radiation, stability_parameter, stability_correction, surface_conditions,
#     surface_conductance, roughness_parameters, wind_profile, ET_to_LE, ms_to_mol,
#     mol_to_ms, flux_to_ustar, CO2_to_umol, umol_to_CO2
# )
# from bigleaf.dummy_data import generate_dummy_data

def test_bigleaf_functions():
    # Generate dummy data
    data = generate_dummy_data()
    constants = bigleaf_constants()

    # Site parameters (adjust as needed)
    zr = 40  # Reference height (m)
    zh = 25  # Canopy height (m)
    d = 16.75  # Displacement height (m)
    z0m = 2  # Roughness length for momentum (m)
    LAI = 4  # Leaf area index
    leafwidth = 0.05  # Leaf width (m)
    Dl = 0.05  # Leaf dimension (m)
    fc = 0.8  # Fractional canopy coverage

    print("Testing Part 1 Functions...")
    # Test WUE_metrics
    wue = WUE_metrics(data, GPP="GPP", NEE="NEE", LE="LE", VPD="VPD", Tair="Tair", constants=constants)
    print("WUE_metrics:", wue)

    # Test Gb_Thom
    gb_thom = Gb_Thom(data['ustar'], constants=constants)
    print("Gb_Thom (first row):", gb_thom.iloc[0])

    # Test Gb_Choudhury
    gb_choudhury = Gb_Choudhury(data, Tair="Tair", pressure="pressure", wind="wind", ustar="ustar",
                                H="H", leafwidth=leafwidth, LAI=LAI, zh=zh, zr=zr, d=d, z0m=z0m,
                                constants=constants)
    print("Gb_Choudhury (first row):", gb_choudhury.iloc[0])

    # Test Gb_Su
    gb_su = Gb_Su(data, Tair="Tair", pressure="pressure", ustar="ustar", wind="wind", H="H",
                  zh=zh, zr=zr, d=d, z0m=z0m, Dl=Dl, fc=fc, LAI=LAI, constants=constants)
    print("Gb_Su (first row):", gb_su.iloc[0])

    # Test aerodynamic_conductance
    aero_cond = aerodynamic_conductance(data, Tair="Tair", pressure="pressure", wind="wind",
                                        ustar="ustar", H="H", zr=zr, zh=zh, d=d, z0m=z0m,
                                        Dl=Dl, fc=fc, LAI=LAI, constants=constants)
    print("Aerodynamic_conductance (first row):", aero_cond.iloc[0])

    # Test roughness_length_heat
    z0h = roughness_length_heat(z0m, aero_cond['kB_h'])
    print("Roughness_length_heat (first row):", z0h[0])

    print("\nTesting Part 2 Functions...")
    # Test intercellular_CO2
    ci = intercellular_CO2(data, Ca="Ca", GPP="GPP", Gs="Gs_mol", constants=constants)
    print("Intercellular_CO2 (first row):", ci[0])

    # Test photosynthetic_capacity
    photo_cap = photosynthetic_capacity(data, C3=True, Temp="Tair", GPP="GPP", Ci="Ci",
                                       PPFD="PPFD", constants=constants)
    print("Photosynthetic_capacity:", photo_cap)

    # Test Arrhenius_temp_response
    Vcmax = (data['GPP'] * (data['Ci'] + 404.9 * (1 + 0.21 / 278.4)) / (data['Ci'] - 42.75))
    arrhenius = Arrhenius_temp_response(Vcmax, data['Tair'], Ha=65.33, Hd=200, dS=0.635, constants=constants)
    print("Arrhenius_temp_response (first row):", arrhenius[0])

    # Test decoupling
    omega = decoupling(data, Tair="Tair", pressure="pressure", Ga="Ga_h", Gs="Gs_ms",
                      LAI=LAI, constants=constants)
    print("Decoupling (first row):", omega[0])

    # Test longwave_conductance
    gr = longwave_conductance(data['Tair'], LAI, constants)
    print("Longwave_conductance (first row):", gr[0])

    # Test Esat_slope
    esat = Esat_slope(data['Tair'], constants=constants)
    print("Esat_slope (first row):", esat.iloc[0])

    # Test psychrometric_constant
    gamma = psychrometric_constant(data['Tair'], data['pressure'], constants)
    print("Psychrometric_constant (first row):", gamma[0])

    # Test VPD_to_rH
    rh = VPD_to_rH(data['VPD'], data['Tair'], constants=constants)
    print("VPD_to_rH (first row):", rh[0])

    print("\nTesting Part 3 Functions...")
    # Test biochemical_energy
    bio_energy = biochemical_energy(data, GPP="GPP", Reco="Reco", constants=constants)
    print("Biochemical_energy (first row):", bio_energy.iloc[0])

    # Test energy_use_efficiency
    eue = energy_use_efficiency(data, Rn="Rn", GPP="GPP", Reco="Reco", constants=constants)
    print("Energy_use_efficiency:", eue)

    # Test energy_closure
    closure = energy_closure(data, Rn="Rn", LE="LE", H="H", G="G", S="S")
    print("Energy_closure:", closure)

    # Test isothermal_net_radiation
    iso_rad = isothermal_net_radiation(data, Tair="Tair", Rn="Rn", constants=constants)
    print("Isothermal_net_radiation (first row):", iso_rad.iloc[0])

    # Test potential_ET
    pet = potential_ET(data, Tair="Tair", pressure="pressure", Rn="Rn", G="G", S="S", constants=constants)
    print("Potential_ET (first row):", pet.iloc[0])

    # Test equilibrium_imposed_ET
    eq_et = equilibrium_imposed_ET(data, Tair="Tair", pressure="pressure", Rn="Rn", G="G", S="S",
                                   VPD="VPD", Ga="Ga_h", constants=constants)
    print("Equilibrium_imposed_ET (first row):", eq_et.iloc[0])

    # Test reference_ET
    ref_et = reference_ET(data, Tair="Tair", pressure="pressure", Rn="Rn", G="G", VPD="VPD",
                          ustar="ustar", zr=zr, zh=zh, d=d, z0m=z0m, constants=constants)
    print("Reference_ET (first row):", ref_et.iloc[0])

    # Test filter_data
    filtered = filter_data(data, filter_growseas=True, filter_precip=True, filter_ustar=True,
                           filter_vars=['GPP', 'NEE'], constants=constants)
    print("Filtered_data (rows):", len(filtered))

    # Test air_density
    rho = air_density(data['Tair'], data['pressure'], constants)
    print("Air_density (first row):", rho[0])

    # Test pressure_from_elevation
    press_elev = pressure_from_elevation(data['elevation'], data['Tair'], constants)
    print("Pressure_from_elevation (first row):", press_elev[0])

    # Test kinematic_viscosity
    viscosity = kinematic_viscosity(data['Tair'], data['pressure'], constants)
    print("Kinematic_viscosity (first row):", viscosity[0])

    # Test Reynolds_Number
    re = Reynolds_Number(data['Tair'], data['pressure'], data['ustar'], hs=0.01, constants=constants)
    print("Reynolds_Number (first row):", re[0])

    # Test optimum_temperature
    topt = optimum_temperature(data, GPP="GPP", Tair="Tair")
    print("Optimum_temperature:", topt)

    # Test potential_radiation
    rad = potential_radiation(data['timestamp'], data['lat'][0], data['lon'][0], data['elevation'][0], constants=constants)
    print("Potential_radiation (first row):", rad.iloc[0])

    # Test stability_parameter
    zeta = stability_parameter(data, Tair="Tair", pressure="pressure", ustar="ustar", H="H",
                              zr=zr, d=d, constants=constants)
    print("Stability_parameter (first row):", zeta[0])

    # Test stability_correction
    stab_corr = stability_correction(zeta)
    print("Stability_correction (first row):", stab_corr.iloc[0])

    # Test surface_conditions
    surf_cond = surface_conditions(data, Tair="Tair", pressure="pressure", Rn="Rn", LE="LE",
                                  H="H", Ga="Ga_h", constants=constants)
    print("Surface_conditions (first row):", surf_cond.iloc[0])

    # Test surface_conductance
    surf_cond = surface_conductance(data, Tair="Tair", pressure="pressure", VPD="VPD", LE="LE",
                                   Ga="Ga_h", constants=constants)
    print("Surface_conductance (first row):", surf_cond.iloc[0])

    # Test roughness_parameters
    rough = roughness_parameters(method="wind_profile", zh=zh, zr=zr, d=d, z0m=z0m, data=data,
                                Tair="Tair", pressure="pressure", wind="wind", ustar="ustar", H="H",
                                constants=constants)
    print("Roughness_parameters:", rough)

    # Test wind_profile
    wind_z = wind_profile(data, z=zh, Tair="Tair", pressure="pressure", ustar="ustar", H="H",
                         wind="wind", zr=zr, zh=zh, d=d, z0m=z0m, constants=constants)
    print("Wind_profile (first row):", wind_z[0])

    # Test unit conversions
    le = ET_to_LE(data['LE'] / latent_heat_vaporization(data['Tair']), data['Tair'])
    print("ET_to_LE (first row):", le[0])

    gs_mol = ms_to_mol(data['Gs_ms'], data['Tair'], data['pressure'], constants)
    print("ms_to_mol (first row):", gs_mol[0])

    gs_ms = mol_to_ms(data['Gs_mol'], data['Tair'], data['pressure'], constants)
    print("mol_to_ms (first row):", gs_ms[0])

    ustar_flux = flux_to_ustar(data['ustar']**2 * air_density(data['Tair'], data['pressure'], constants),
                              data['Tair'], data['pressure'], data['wind'], constants)
    print("flux_to_ustar (first row):", ustar_flux[0])

    co2_umol = CO2_to_umol(data['Ca'], data['Tair'], data['pressure'], constants)
    print("CO2_to_umol (first row):", co2_umol[0])

    co2_ppm = umol_to_CO2(co2_umol, data['Tair'], data['pressure'], constants)
    print("umol_to_CO2 (first row):", co2_ppm[0])

if __name__ == "__main__":
    test_bigleaf_functions()

Testing Part 1 Functions...
WUE_metrics: {'WUE': np.float64(4.559919357977501), 'WUE_NEE': np.float64(inf), 'IWUE': np.float64(8.365074511602046), 'uWUE': np.float64(6.536030714865674)}
Gb_Thom (first row): Gb_h       0.097543
Rb_h      10.251849
kB_h       1.977580
Gb_CO2     0.074106
Name: 0, dtype: float64
Gb_Choudhury (first row): Gb_h      0.118694
Rb_h      8.425020
kB_h      1.625185
Gb_CO2    0.090175
Name: 0, dtype: float64
Gb_Su (first row): Gb_h      0.114505
Rb_h      8.733234
kB_h      1.684639
Gb_CO2    0.086992
Name: 0, dtype: float64
Aerodynamic_conductance (first row): Ga_m       0.071420
Ra_m      14.001641
Ga_h       0.041231
Ra_h      24.253491
Gb_h       0.097543
Rb_h      10.251849
kB_h       1.977580
z0h        0.276808
zeta            NaN
psi_h           NaN
Ra_CO2    27.495814
Ga_CO2     0.036369
Gb_CO2     0.074106
Name: 0, dtype: float64
Roughness_length_heat (first row): 0.27680760482338185

Testing Part 2 Functions...
Respiration from the leaves is ignored 

## Run test in R

In [10]:
%%R

# Load required libraries
library(bigleaf)
library(lubridate)

# Generate dummy data (same as Python generate_dummy_data)
generate_dummy_data <- function() {
  set.seed(123)

  # Define parameters
  n_timesteps <- 1440  # 30 days * 48 half-hourly timesteps
  timestep <- 30 * 60  # 30 minutes in seconds
  start_time <- ymd_hms("2023-01-01 00:00:00")
  time <- seq(from = start_time, by = timestep, length.out = n_timesteps)

  # Create diurnal cycle
  diurnal <- cos(2 * pi * seq(0, n_timesteps - 1) / 48)

  # Initialize data frame
  data <- data.frame(
    timestamp = time,
    Tair = numeric(n_timesteps),
    pressure = numeric(n_timesteps),
    wind = numeric(n_timesteps),
    ustar = numeric(n_timesteps),
    H = numeric(n_timesteps),
    GPP = numeric(n_timesteps),
    NEE = numeric(n_timesteps),
    LE = numeric(n_timesteps),
    VPD = numeric(n_timesteps),
    PPFD = numeric(n_timesteps),
    Ca = numeric(n_timesteps),
    Gs_mol = numeric(n_timesteps),
    Gs_ms = numeric(n_timesteps),
    Ci = numeric(n_timesteps),
    Ga_h = numeric(n_timesteps)
  )

  # Generate dummy data
  data$Tair <- 20 + 5 * diurnal + 2 * rnorm(n_timesteps)
  data$pressure <- 100 + 2 * rnorm(n_timesteps)
  data$wind <- 3 + 1 * abs(diurnal) + 0.5 * rnorm(n_timesteps)
  data$ustar <- 0.3 + 0.1 * abs(diurnal) + 0.05 * rnorm(n_timesteps)
  data$H <- 125 + 100 * diurnal + 25 * rnorm(n_timesteps)
  data$GPP <- pmax(0, 15 + 10 * (-diurnal) + 3 * rnorm(n_timesteps))
  data$NEE <- -5 + 7.5 * diurnal + 3 * rnorm(n_timesteps)
  data$LE <- pmax(0, 200 + 150 * (-diurnal) + 50 * rnorm(n_timesteps))
  data$VPD <- pmax(0.5, 1.75 + 0.75 * (-diurnal) + 0.3 * rnorm(n_timesteps))
  data$PPFD <- pmax(0, 1000 * (-diurnal) + 200 * rnorm(n_timesteps))
  data$Ca <- 400 + 10 * rnorm(n_timesteps)
  data$Gs_mol <- pmax(0, 0.25 + 0.15 * (-diurnal) + 0.05 * rnorm(n_timesteps))
  R <- 8.314
  T <- 20 + 273.15
  P <- 100 * 1000
  data$Gs_ms <- data$Gs_mol * (R * T / P)
  data$Ci <- 275 + 50 * rnorm(n_timesteps)
  data$Ga_h <- pmax(0.01, 0.055 + 0.03 * rnorm(n_timesteps))

  # Introduce 5% missing values
  introduce_missing <- function(x) {
    n <- length(x)
    missing <- sample(1:n, size = round(0.05 * n))
    x[missing] <- NA
    return(x)
  }

  for (col in names(data)[-1]) {
    data[[col]] <- introduce_missing(data[[col]])
  }

  return(data)
}

# Test bigleaf functions
test_bigleaf_functions <- function() {
    # Generate dummy data
    # data <- generate_dummy_data()
    # Read dummy data
    data <- read.csv("dummy_bigleaf_data.csv")

    # # Define parameters (adjust as needed)
    # zr <- 20.0    # Reference height (m)
    # zh <- 15.0    # Canopy height (m)
    # d <- 10.0     # Displacement height (m)
    # z0m <- 1.5    # Roughness length (m)
    # Dl <- 0.05    # Leaf dimension (m)
    # fc <- 0.8     # Fractional canopy coverage
    # LAI <- 3.0    # Leaf area index
    # leafwidth <- 0.05  # Leaf width (m)

    # Site parameters (same as Python)
    zr <- 40  # Reference height (m)
    zh <- 25  # Canopy height (m)
    d <- 16.75  # Displacement height (m)
    z0m <- 2  # Roughness length for momentum (m)
    LAI <- 4  # Leaf area index
    leafwidth <- 0.05  # Leaf width (m)
    Dl <- 0.05  # Leaf dimension (m)
    fc <- 0.8  # Fractional canopy coverage

    # Get constants
    constants <- bigleaf.constants()

    # Test 1: WUE.metrics (equivalent to Python WUE_metrics)
    wue <- WUE.metrics(data, GPP="GPP", NEE="NEE", LE="LE", VPD="VPD", Tair="Tair")
    cat("WUE.metrics:\n")
    print(wue)

    # Test 2: boundary.layer.conductance with Rb.model="Thom_1972" (equivalent to Gb_Thom)
    gb_thom <- Gb.Thom(data$ustar)
    cat("boundary.layer.conductance (Thom_1972, first row):\n")
    print(head(gb_thom, 1))

    # Test 3: boundary.layer.conductance with Rb.model="Choudhury_1986" (equivalent to Gb_Choudhury)
    gb_choudhury <- Gb.Choudhury(data, Tair="Tair", pressure="pressure", wind="wind",
                                ustar="ustar", H="H", leafwidth=leafwidth,
                                LAI=LAI, zh=zh, zr=zr, d=d, z0m=z0m)
    cat("boundary.layer.conductance (Choudhury_1986, first row):\n")
    print(head(gb_choudhury, 1))

    # Test 4: boundary.layer.conductance with Rb.model="Su_2001" (equivalent to Gb_Su)
    gb_su <- Gb.Su(data, Tair="Tair", pressure="pressure", wind="wind",
                                ustar="ustar", H="H", zh=zh, zr=zr, d=d, z0m=z0m,
                                Dl=Dl, fc=fc, LAI=LAI)
    cat("boundary.layer.conductance (Su_2001, first row):\n")
    print(head(gb_su, 1))

    # Test 5: aerodynamic.conductance (equivalent to aerodynamic_conductance)
    # Note: R bigleaf includes CO2 conductance by default, so no need for Sc, Sc_name
    aero_cond <- aerodynamic.conductance(data, Tair="Tair", pressure="pressure", wind="wind",
                                        ustar="ustar", H="H", zr=zr, zh=zh, d=d, z0m=z0m,
                                        Dl=Dl, fc=fc, LAI=LAI)
    cat("aerodynamic.conductance (first row):\n")
    print(head(aero_cond, 1))

    # Test 6: intercellular.CO2 (equivalent to intercellular_CO2)
    ci <- intercellular.CO2(data, Ca="Ca", GPP="GPP", Gs="Gs_mol")
    cat("intercellular.CO2 (first 5 values):\n")
    print(head(ci, 5))

    # Test 7: photosynthetic.capacity (equivalent to photosynthetic_capacity)
    photo <- photosynthetic.capacity(data, C3=TRUE, Temp="Tair", GPP="GPP", Ci="Ci", PPFD="PPFD")
    cat("photosynthetic.capacity:\n")
    print(photo)

    # Test 8: decoupling (equivalent to decoupling)
    omega <- decoupling(data, Tair="Tair", pressure="pressure", Ga="Ga_h", Gs="Gs_ms", LAI=LAI)
    cat("decoupling (first 5 values):\n")
    print(head(omega, 5))

    # Test 9: Esat.slope (equivalent to Esat_slope)
    esat <- Esat.slope(data$Tair)
    cat("Esat.slope (first row):\n")
    print(head(esat, 1))

    # Test 10: psychrometric.constant (equivalent to psychrometric_constant)
    gamma <- psychrometric.constant(data$Tair, data$pressure)
    cat("psychrometric.constant (first 5 values):\n")
    print(head(gamma, 5))

    # Test 11: VPD.to.rH (equivalent to VPD_to_rH)
    rh <- VPD.to.rH(data$VPD, data$Tair)
    cat("VPD.to.rH (first 5 values):\n")
    print(head(rh, 5))

    wind_z = wind.profile(data, z=zh, Tair="Tair", pressure="pressure", ustar="ustar", H="H",
                         wind="wind", zr=zr, zh=zh, d=d, z0m=z0m)
    print(head(wind_z, 1))
}

# Run the test
test_bigleaf_functions()

WUE.metrics:
     WUE  WUE_NEE     IWUE     uWUE 
2.777748      Inf 5.201163 3.880341 
boundary.layer.conductance (Thom_1972, first row):
        Gb_h     Rb_h    kB_h     Gb_CO2
1 0.09633099 10.38088 1.96527 0.07318498
boundary.layer.conductance (Choudhury_1986, first row):
       Gb_h     Rb_h     kB_h     Gb_CO2
1 0.1175864 8.504382 1.610019 0.08933327
boundary.layer.conductance (Su_2001, first row):
       Gb_h     Rb_h     kB_h     Gb_CO2
1 0.1135636 8.805639 1.667052 0.08627701
Note new column 'z0h' in the function output for 'bigleaf' versions > 0.7.5.
aerodynamic.conductance (first row):
        Ga_m     Ra_m       Ga_h     Ra_h       Gb_h     Rb_h    kB_h       z0h
1 0.08019433 12.46971 0.04376256 22.85059 0.09633099 10.38088 1.96527 0.2802361
  zeta psi_h   Ra_CO2     Ga_CO2     Gb_CO2
1   NA    NA 26.13372 0.03826475 0.07318498
Respiration from the leaves is ignored and set to 0.
intercellular.CO2 (first 5 values):
[1] 405.8008 388.7033 405.8434 409.1359 392.5108
Respiration


Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union

In addition: Warning message:
In aerodynamic.conductance(data, Tair = "Tair", pressure = "pressure",  :
  Provided roughness length parameters (zr,d,z0m) are not used if 'wind_profile = FALSE' (the default). Ra_m is calculated as wind / ustar^2
